In [ ]:
"""
Author: Hanshuo Wu
This file is for the voxelization of point cloud and the quantification of different materials.
"""

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt 

In [2]:
color_code = np.array([[220,220,220],  # white - class 0
                       [250,0,0], # red - Rustication - class 1
                       [0,250,0], # green - Brick - class 2
                       [0,0,250], # blue - Stucco  - class 3
                       [250,250,0], # yellow - Wood  - class 4
                      ])

In [33]:
# Initialization
pts_pred = np.load("results/scan1-0829.npy")
result = pts_pred

original_pc = o3d.geometry.PointCloud()
original_pc.points = o3d.utility.Vector3dVector(pts_pred[:,:3])
original_pc.colors = o3d.utility.Vector3dVector(pts_pred[:,3:6]/255)

pred_pc = o3d.geometry.PointCloud()
pred_pc.points = o3d.utility.Vector3dVector(pts_pred[:,:3])
pts_pred[:,3:6] = np.asarray([color_code[int(i)] for i in pts_pred[:,6]])
pred_pc.colors = o3d.utility.Vector3dVector(pts_pred[:,3:6]/255)

voxel_size = 0.8

In [34]:
o3d.visualization.draw_geometries([original_pc])
o3d.visualization.draw_geometries([o3d.geometry.VoxelGrid.create_from_point_cloud(original_pc,    #Voxelized Original PC
                                                            voxel_size=voxel_size)])
o3d.visualization.draw_geometries([pred_pc])  #Predicted PC
o3d.visualization.draw_geometries([o3d.geometry.VoxelGrid.create_from_point_cloud(pred_pc,    #Voxelized Predicted PC
                                                            voxel_size=voxel_size)])   

In [35]:
# Get Voxel Grid
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pred_pc,
                                                            voxel_size=voxel_size)
print(voxel_grid)
#o3d.visualization.draw_geometries([voxel_grid])

VoxelGrid with 6477 voxels.


In [36]:
%%time
# Get tuples: (Voxel Index, Predict Result)
predict_list = []
for i in pts_pred:
    idx = voxel_grid.get_voxel(np.array([i[0],i[1],i[2]]))
    predict = int(i[6])
    predict_list.append((tuple(idx),predict))

CPU times: total: 188 ms
Wall time: 412 ms


In [37]:
%%time
# Get matrix of voxel grids, in which each vo contains the SET of predict results

# Create a voxel grid matrix, with shape of max_X*max_Y*max_Z
matrix = np.empty(
    tuple(np.max(np.asarray([i[0] for i in predict_list]),axis=0)+1)
    , dtype=object)    

# For each point in a voxel, add class of the point to the matrix corresponding to the voxel.
for i in predict_list:
    if matrix[i[0]] is None: 
        matrix[i[0]] = set()    
    matrix[i[0]].add(i[1])
    #print(i[0], matrix[i[0]])

# Count each class
count = np.array([],dtype="int32")
for idx, x in np.ndenumerate(matrix):
    if x is not None:
        count = np.hstack((count,np.asarray(list(x))))
print("Class Counts: ",np.bincount(count))
print("Voxel Size =", voxel_grid.voxel_size)

Class Counts:  [4350  316 2006  129]
Voxel Size = 0.8
CPU times: total: 78.1 ms
Wall time: 264 ms


In [38]:
%%time
# Get matrix of voxel grids, in which each cell is only the MAJORITY of results
matrix = np.empty(
    tuple(np.max(np.asarray([i[0] for i in predict_list]),axis=0)+1)
    , dtype=object) 
empty = set("1")
for i in predict_list:
    if matrix[i[0]] is None: 
        matrix[i[0]] = list()   
    matrix[i[0]].append(i[1])
    #print(i[0], matrix[i[0]])
    
for idx, x in np.ndenumerate(matrix):
    if x is not None:
        matrix[idx] = np.argmax(np.bincount(x))   # Majority Voting
        #matrix[idx] = stats.mode(nums)[0][0]
        
# Count each class
count=np.array([],dtype="int32")
for idx, x in np.ndenumerate(matrix):
    if x is not None:
        count=np.append(count,[x])
print("Class Counts: ",np.bincount(count))
print("Voxel Size =", voxel_grid.voxel_size)

Class Counts:  [4240  221 1892  124]
Voxel Size = 0.8
CPU times: total: 234 ms
Wall time: 327 ms


In [39]:
# Visualization of one class, after majority voting
bounding_box = []
for idx, x in np.ndenumerate(matrix):
    if x == 1:
        bb = o3d.geometry.AxisAlignedBoundingBox.create_from_points(voxel_grid.get_voxel_bounding_points(idx))
        bb.color = np.array([0,0,0])  # feel free to choose any other color
        bounding_box.append(bb)
#axes = o3d.geometry.TriangleMesh.create_coordinate_frame(origin=np.array([0,0,15]), size=10)  # in order to be sure of the direction
#o3d.visualization.draw_geometries([voxel_grid, *bounding_box, axes])
o3d.visualization.draw_geometries([voxel_grid, *bounding_box])

In [ ]:
original_pc.estimate_normals()
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        original_pc, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])